In [3]:
%use s2, algoquant

val linearShrinkageMeanEstimatorParam = 0.5
val linearShrinkageCovEstimatorParam = 0.5
val riskAverse = 1.0
val lowBoundValue = 0.0
val upBoundValue = 1.0
val csvdata = CsvCacheProvider("sample_assets.csv", Currencies.USD)
val simInterval = csvdata.dataInterval()
val rebPer = Period.parse("P1M")
val timeWin = Period.parse("P6M")
val sim = OptimizationAlgorithmComparisonSimulation(
    csvdata.allProducts(simInterval),
    csvdata,
    csvdata,
    rebPer,
    timeWin
)
// long only constraints
val stocksCache = BufferCache<List<String>>()
val stockTics = mutableListOf<String>()
csvdata.allProducts(csvdata.dataInterval()).forEach { p ->
    stockTics.add(p.symbol())
}
stocksCache.add(csvdata.dataInterval().start(),stockTics)
val constraint = MarkowitzLongOnlyConstraint().addLongOnlyPositionLimit(stocksCache)

val constraintFactories = ArrayList<SOCPPortfolioConstraintFactory>()
constraintFactories.add(SOCPUnityFactory())

val lowBoundCache = BufferCache<Map<String, Double>>()
val upBoundCache = BufferCache<Map<String, Double>>()


for (timedEntry in stocksCache) {
    val tempDate = timedEntry.time()
    val lowBound = HashMap<String, Double>()
    val upBound = HashMap<String, Double>()
    for (ticker in timedEntry.data()){
        lowBound[ticker] = lowBoundValue
        upBound[ticker] = upBoundValue
    }
    lowBoundCache.add(tempDate, lowBound)
    upBoundCache.add(tempDate, upBound)
}
constraintFactories.add(SOCPLowerLimitFactory(lowBoundCache))
constraintFactories.add(SOCPUpperLimitFactory(upBoundCache))


// optimizer

val optimAlgo = SOCPDynamicContraintOptimizationAlgorithm(riskAverse, constraintFactories)
// estimators
optimAlgo.meanEstimator = MeanAnnualizingWrapper(LinearShrinkageMeanEstimator(linearShrinkageMeanEstimatorParam), rebPer)
optimAlgo.covarianceEstimator = CovarianceAnnualizingWrapper(LinearShrinkageCovarianceEstimator(linearShrinkageCovEstimatorParam), rebPer)
// run simulation
sim.addAlgorithm("simulation", optimAlgo)
sim.setRiskFreeRateLookup(DbRiskFreeRateLookup().newRiskFreeRateLookup(simInterval))
sim.setBenchmarkLookupFactory("SP500", BenchmarkLookupFactory())

val result = sim.run(simInterval)
println("success")

07:34:08.104 [main] INFO  org.reflections.Reflections,239 - Reflections took 27 ms to scan 1 urls, producing 3 keys and 13 values 
07:34:10.349 [main] INFO  t.n.a.m.p.s.OptimizationAlgorithmComparisonSimulation,189 - Running algorithm 'simulation' (window=P6M, rebalance=P1M)...
07:34:12.154 [main] INFO  t.n.a.m.s.r.RebalancePortfolio,135 - Calculating weights for rebalancing point at 2016-07-29T00:00
07:34:12.163 [main] WARN  t.n.a.m.s.r.RebalancePortfolio,156 - failed to compute new positions: end of available data
07:34:12.167 [main] INFO  t.n.a.m.s.r.RebalancePortfolio,135 - Calculating weights for rebalancing point at 2016-08-29T00:00
07:34:12.168 [main] WARN  t.n.a.m.s.r.RebalancePortfolio,156 - failed to compute new positions: end of available data
07:34:12.171 [main] INFO  t.n.a.m.s.r.RebalancePortfolio,135 - Calculating weights for rebalancing point at 2016-09-29T00:00
07:34:12.172 [main] WARN  t.n.a.m.s.r.RebalancePortfolio,156 - failed to compute new positions: end of availab